In [2]:
import pandas as pd 
import numpy as np

# Amalagamation
The main idea is to do an amalgamation per variable: i.e. the predictions of all architecture at time point t and take the average of those. This begs the question of how to combine them. For example LSTM1/CNN1 does not have the same structure as FNN1. 

Amalgamtion of the all models should be doable, things get more complicated once we start looking at all the variables.

* Amalagamation MEV
* Amalgamation TA
* Amalgamtion ALL 
* Amalgamation PCA MEV
* Amalgamation PCA TA
* Amalgamation PCA ALL
* Amalgamtion per MEV variable 
* Amalgamation per TA variable 

In [80]:
results = pd.read_excel(open('output/ALL.xlsx', 'rb'), sheet_name='MEV Variables', engine='openpyxl', index_col=0)
variables = results.Dataset.unique()
models = results.Method.unique()
table = pd.DataFrame(index=variables)
architectures = ['CNN', 'MLP'] #TODO: Incomplete
dataset = 'MEV'
aggregatedDict = dict()

In [83]:
"""
The main idea is that I try to aggregate the predictions over all architectures with all hidden units for a single variable. Thus aggregatedict is a dictionairy with dataframe of all the predictions per varaible.
Thus aggregatedDict{'DP'} would yield a dataframe where each column is the prediction vector of a architecture. Aka, when you take the row wise average you have an amalgamation for a variable based on all models. 
"""
for architecture in architectures:
    # Set the hidden unit definition depending on the architecture
    if(architecture == 'CNN' or architecture == 'RF'):
        hidden_sizes = None
    elif(architecture == 'MLP' or architecture == 'LSTM'):
        hidden_sizes = [32, 16, 8, 4, 2]
    else:
        hidden_sizes = [[32], [32, 16], [32, 16, 8], [32, 16, 8, 4], [32, 16, 8, 4, 2]] 
        
    for variable in variables:
        variable = variable.replace(dataset+': ', '')
        if(hidden_sizes is not None):
            for hidden in hidden_sizes:
                #For each variation of a certain variable read the relevant file and concatenate the predictions to the dataframe stored in the aggrefatedDict dictionairy for said variable.
                results = pd.read_parquet('output/' + str(architecture) + '_' + str(dataset) +'_' + str(hidden).replace('[', '').replace(']', '').replace(', ', '_') + '_' + str(variable).replace(' ', '').replace('%', '') + '.gzip')
                try:
                    df = pd.concat([aggregatedDict.get(variable), results.Pred], axis = 1)
                except:
                    df = results.Pred
                aggregatedDict.update({variable: df})
        
        #For the architectures withouth hidden units, those are irrelevant. 
        elif(hidden_sizes is None):
            results = pd.read_parquet('output/' + str(architecture) + '_' + str(dataset) +'_' + str(variable).replace(' ', '').replace('%', '') + '.gzip')
            try:
                df = pd.concat([aggregatedDict.get(variable), results.Pred], axis = 1)
            except:
                df = results.Pred
            aggregatedDict.update({variable: df})
            
   

In [84]:
aggregatedDict.get('DP')

,Pred,Pred,Pred,Pred,Pred,Pred,Pred,Pred,Pred,Pred,Pred,Pred
0,-0.143620,0.115406,0.098237,0.030704,0.123148,0.045416,-0.143620,0.115406,0.098237,0.030704,0.123148,0.045416
1,-0.151327,0.115406,0.098237,0.030704,0.123148,0.045303,-0.151327,0.115406,0.098237,0.030704,0.123148,0.045303
2,-0.138427,0.115406,0.098237,0.030704,0.123148,0.045485,-0.138427,0.115406,0.098237,0.030704,0.123148,0.045485
3,0.038207,0.115327,0.098237,0.026551,0.123148,0.047351,0.038207,0.115327,0.098237,0.026551,0.123148,0.047351
4,0.136788,0.115187,0.098237,0.019520,0.123148,0.049509,0.136788,0.115187,0.098237,0.019520,0.123148,0.049509
...,...,...,...,...,...,...,...,...,...,...,...,...
643,0.007399,-0.007217,-0.045582,-0.021088,0.385860,-0.319547,0.007399,-0.007217,-0.045582,-0.021088,0.385860,-0.319547
644,0.006972,-0.007217,-0.048098,-0.021088,0.377980,-0.308679,0.006972,-0.007217,-0.048098,-0.021088,0.377980,-0.308679
645,0.008673,-0.007217,-0.038085,-0.021088,0.409342,-0.351932,0.008673,-0.007217,-0.038085,-0.021088,0.409342,-0.351932
646,0.007968,-0.007217,-0.042238,-0.021088,0.396336,-0.333994,0.007968,-0.007217,-0.042238,-0.021088,0.396336,-0.333994
